In [ ]:
import pandas as pd 
from sklearn.linear_model import LinearRegression
from itertools import chain
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

Realizando o tratamento da estrutura do dataframe

In [ ]:
#criando o dataframe
anos=['1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005',
'2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017',
'2018','2019','2020','2021']
dataset=pd.DataFrame()
for i in anos:
    tabela=pd.read_excel('arrecadação.xlsx',sheet_name=i)
    dataset=pd.concat([dataset,tabela],axis=0,ignore_index=True)
dataset

O arquivo possui várias planilhas, caso queira uma particular basta fazer: pd.read_excel('arrecadação.xlsx',sheet_name='nome da planilha')
caso queira todas, defina: sheet_name=none, mas o arquivo passará a ser um dicionário. 

In [ ]:
#deletando dos valores nulos
dataset=dataset.drop(dataset.index[[0,1,2,3]])
dataset.head()

In [ ]:
#renomeando as colunas
# colunas=['receitas','janeiro','fevereiro','março','abril','maio','junho','julho','agosto','setembro'
# ,'outubro','novembro','dezembro','total']
dataset.rename(columns={'Unnamed: 0':'receitas','Unnamed: 1': 'JAN','Unnamed: 2':'FEV',
'Unnamed: 3':'MAR','Unnamed: 4':'ABR','Unnamed: 5':'MAI','Unnamed: 6':'JUN','Unnamed: 7':'JUL',
'Unnamed: 8':'AGO','Unnamed: 9':'SET','Unnamed: 10':'OUT','Unnamed: 11':'NOV',
'Unnamed: 12':'DEZ','Unnamed: 13':'total'},inplace=True)
dataset.head()

In [ ]:
#apagando a primeira linha; ambiguidade
dataset=dataset.drop(dataset.index[[0]])
dataset

Tratamento dos dados

In [ ]:
#Vamos juntar os imposto de renda das pessoas físicas e juridicas no intervalo de tempo: 1994-2021
#A frequência dos dados IR PF E PF: 43, logo, temos uma P.A
# termos iniciais: IR-PF:9;IR-PJ:10.
#termo final: 1127
periodo_pessoafisica=[]
lista_pessoajuridica=[]
valores_pessoafisica=[]
valores_pessoajuridica=[]

#Filtro dos elementos
for x in range(9,1127,43):
    periodo_pessoafisica.append(x) #trabalhando nossa P.A IR-pessoa fisica
for j in periodo_pessoafisica: 
    valores_pessoafisica.append(dataset.iloc[j,0:13].values) #adicionando os valores
for p in range(10,1127,43):
    lista_pessoajuridica.append(p) #processo análogo ao IR-PF
for z in lista_pessoajuridica: 
    valores_pessoajuridica.append(dataset.iloc[z,0:13].values) #análogo

#transformando uma lista multinível em unidimensional
valores_pessoafisica=list(chain(*valores_pessoafisica)) 
valores_pessoajuridica=list(chain(*valores_pessoajuridica))
#excluindo a string
k='  I.RENDA-PESSOA FÍSICA'
j='  I.RENDA-PESSOA JURÍDICA'
while(k in valores_pessoafisica):
    valores_pessoafisica.remove(k)
while(j in valores_pessoajuridica):
    valores_pessoajuridica.remove(j)
#criando o data frame
imposto=pd.DataFrame()
imposto.insert(0,'pessoafisica',valores_pessoafisica)
imposto.insert(1,'pessoajuridica',valores_pessoajuridica)
imposto

In [ ]:
#correlação
imposto.corr().style.background_gradient(cmap='Blues')

In [ ]:
#Regressão linear
x=imposto.iloc[:,0].values
y=imposto.iloc[:,1].values
#modelo de regressão
X=x.reshape(-1,1)
Y=y.reshape(-1,1)
modelo=LinearRegression()
modelo.fit(X,Y)

In [ ]:
#score
modelo.score(X,Y)

In [ ]:
#previsão
previsao=modelo.predict(X)

In [ ]:
#dispersão
plt.scatter(X,Y,c='green')
plt.title('Dispersão IR')
plt.xlabel('IR-PF')
plt.ylabel('IR-PJ')
plt.plot(X,previsao,color='red')

In [ ]:
#IR- pessoa física
sns.distplot(imposto.iloc[:,0].values,bins=10,axlabel='IR-Pessoa física').set_title('Imposto de renda-PF')

In [ ]:
#IR-pessoa juridica
sns.distplot(imposto.iloc[:,1].values,bins=10,axlabel='IR-Pessoa juridica').set_title('Imposto de renda-PJ ')